In [25]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/mnt/nfs/work1/elm/chenyun/PhraseCutPano')
!pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/chenyun/work1/PhraseCutPano


In [26]:
from PhraseCutDataset.utils.vg_loader import VGLoader
vg = VGLoader()

Loader loading nar_count: data/refvg/amt_result/name_att_rel_count_amt.json
Number of categories: 1272 / 3103, frequency thresh: 21 (excluding [INV] [UNK])
Number of attributes: 593 / 12143, frequency thresh: 21 (excluding [INV] [UNK])
Number of relationships: 126 / 3110, frequency thresh: 21 (excluding [INV] [UNK])
Loader loading data_split: data/refvg/image_data_split3000.json
split: train_val_test_miniv
Loader loading scene_graphs_pp_train.json
Loader loading scene_graphs_pp_val.json
Loader loading scene_graphs_pp_test.json
Loader loading scene_graphs_pp_miniv.json
we have 108077 images.
we have 3802374 (filtered) objects, 35.2 per image.
we have 2316104 relations, 21.4 per image.


In [27]:
from PhraseCutDataset.internal.process.vg_processor import VGProcessor

In [33]:
vgp = VGProcessor(vg_loader=vg, split='')

RefVGLoader loading refer data
RefVGLoader loading refer_filtered_instance_miniv.json
RefVGLoader loading refer_filtered_instance_val.json
RefVGLoader loading refer_filtered_instance_test.json
RefVGLoader loading refer_filtered_instance_train.json
RefVGLoader preparing data
RefVGLoader ready.


In [34]:
ms = dict()
failed_tasks = list()
dup_tasks = list()

for img_id, img_tasks in vgp.ImgReferTasks.items():
    for task in img_tasks:
#         if task['task_id'] in skip_tasks:
#             continue
        try:
            modified = vgp.refine_task_data(task, verbose=False)
            ms[task['task_id']] = modified
            ph_words = task['phrase'].split()
            for i in range(len(ph_words) - 1):
                if ph_words[i] == ph_words[i+1]:
                    print(task['task_id'], modified, '"'+task['phrase']+'"','"'+task.get('original_phrase', '')+'"', 
                          task['phrase_structure'])
                    dup_tasks.append(task['task_id'])
                    break
        except Exception as e:
            print(e)
            failed_tasks.append(task['task_id'])
            print('Failed %d.' % len(failed_tasks), task['task_id'], '"'+task['phrase']+'"', task['phrase_structure'])

print(len(failed_tasks))
print(failed_tasks)

ERR: cannot find name for type=verbose
3656__4347083 "white White  shoe on left foot" {'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347083, 'predicate': 'on', 'object_id': 4347084, 'relationship_id': 4700907}], 'relation_descriptions': []}
"white shoe on left foot"
['shoe']

Failed 1. 3656__4347083 "white White  shoe on left foot" {'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347083, 'predicate': 'on', 'object_id': 4347084, 'relationship_id': 4700907}], 'relation_descriptions': []}
ERR: cannot find name for type=verbose
3656__4347087 "white White  shoe" {'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347087, 'predicate': 'on', 'object_id': 4347088, 'relationship_id': 4700909}], 'relation_descriptions': []}
"white shoe"
['shoe']

Failed 2. 3656__4347087 "white White  shoe" {'type': 'verbose', 'name': '', 'attributes': ['whi

In [35]:
remain_tasks = failed_tasks
for img_id, img_tasks in vgp.ImgReferTasks.items():
    for task in img_tasks:
        if task['task_id'] in remain_tasks:
            print(task['task_id'])
            print(task['phrase'])
            print(task['phrase_structure'])
            for ann_id in task['ann_ids']:
                rels = vgp.objects[ann_id]['relations']
                for rel in rels:
                    print(rel['predicate'], vgp.objects[rel['object_id']]['names'])

3656__4347083
white White  shoe on left foot
{'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347083, 'predicate': 'on', 'object_id': 4347084, 'relationship_id': 4700907}], 'relation_descriptions': []}
on ['left foot']
3656__4347087
white White  shoe
{'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347087, 'predicate': 'on', 'object_id': 4347088, 'relationship_id': 4700909}], 'relation_descriptions': []}
on ['right foot']
3656__4347091
white white  lines
{'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347091, 'predicate': 'on', 'object_id': 4347092, 'relationship_id': 4700911}], 'relation_descriptions': []}
on ['ground']
3656__4347100
white white  lines on court
{'type': 'verbose', 'name': '', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347100, 'predicate': 'on', 'object_id': 4347101, 'relationship_id':

In [36]:
print(remain_tasks)
failed_tasks = list()

for img_id, img_tasks in vgp.ImgReferTasks.items():
    for task in img_tasks:
        if task['task_id'] in remain_tasks:
            if task['phrase_structure']['type'] == 'verbose':  # originally they are task['phrase_structure']=={}
                task['phrase_structure'] = dict()
                phrase_words = task['phrase'].lower().split()
                if 'is' in phrase_words:
                    phrase_words.remove('is')
                if 'are' in phrase_words:
                    phrase_words.remove('are')
                if phrase_words[0] == phrase_words[1]:
                    phrase_words.pop(0)
                task['phrase'] = ' '.join(phrase_words)
                print(task['task_id'], task['phrase'])
                try:
                    modified = vgp.refine_task_data(task, verbose=True)
                    ms[task['task_id']] = modified
                    print(task['task_id'], modified)
                    remain_tasks.remove(task['task_id'])
                except Exception as e:
                    print(e)
                    failed_tasks.append(task['task_id'])
                    print('Failed %d.' % len(failed_tasks), task['task_id'], '"'+task['phrase']+'"', 
                          task['phrase_structure'])
                    
print(remain_tasks)

['3656__4347083', '3656__4347087', '3656__4347091', '3656__4347100', '4351__1558186', '2365139__2277684', '1276__1639788', '1276__1639802', '2364991__1864998', '3569__4344249_4344250', '2389716__4485595']
3656__4347083 white shoe on left foot
start refining task 3656__4347083 white shoe on left foot {}
from split_rel_descriptions: "on left foot" [] ['on']
returned rds:  [('on', 'left foot')]
WARNING! type=relation, case=3
3656__4347083 "white shoe on left foot" {'type': 'relation', 'name': 'shoe', 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347083, 'predicate': 'on', 'object_id': 4347084, 'relationship_id': 4700907}], 'relation_descriptions': [('on', 'left foot')]}
ref_ann: {'obj_id': 4347083, 'image_id': 3656, 'box': [160, 374, 121, 61], 'names': ['shoe'], 'attributes': ['white'], 'relations': [{'synsets': [], 'subject_id': 4347083, 'predicate': 'on', 'object_id': 4347084, 'relationship_id': 4700907}]}
st_anns: [{'obj_id': 4347069, 'image_id': 3656, 'box': [1

In [37]:
for i, t in enumerate(vgp.ImgReferTasks[2389716]):
    print(i, t['task_id'])

0 2389716__4485563
1 2389716__4485546
2 2389716__4485559
3 2389716__4485595


In [38]:
t = vgp.ImgReferTasks[2389716][3]
print(t['phrase'])
print(t['ann_ids'])
print(t['phrase_structure'])
for ann_id in t['ann_ids']:
    print(vgp.objects[ann_id])
    rels = vgp.objects[ann_id]['relations']
    for rel in rels:
        print(rel['predicate'], vgp.objects[rel['object_id']]['names'])
# vgp.refine_task_data(t, verbose=True)


snow  white
[4485595]
{'attributes': [], 'type': 'relation', 'name': 'snow', 'relations': [{'relationship_id': 4757436, 'subject_id': 4485583, 'predicate': 'on', 'synsets': [], 'object_id': 4485584}]}
{'obj_id': 4485595, 'image_id': 2389716, 'box': [175, 300, 127, 170], 'names': ['snow'], 'attributes': [], 'relations': [{'synsets': [], 'subject_id': 4485595, 'predicate': '', 'object_id': 4485596, 'relationship_id': 4757442}]}
 ['white']


In [39]:
t = vgp.ImgReferTasks[2389716][3]
t['phrase'] = 'white snow'
t['original_phrase'] = 'snow  white'
t['phrase_structure'] = {'attributes': ['white'], 'type': 'name', 'name': 'snow', 'relations': [], 
                         'relation_descriptions': []}

In [40]:
print(len(dup_tasks))

for img_id, img_tasks in vgp.ImgReferTasks.items():
    for task in img_tasks:
        if task['task_id'] in dup_tasks and len(task['phrase_structure']['relation_descriptions']) > 0:
            print(task['phrase'], '\t', [task['phrase_structure'][k] for k in ['name', 'attributes', 
                                                                               'relation_descriptions']])

28
white apron apron 	 ['white', [], [('apron', 'apron')]]
person riding riding bike 	 ['person riding', [], [('riding', 'bike')]]
cotton long uniform knee high solid red red sock worn on leg 	 ['sock', ['red', 'solid red', 'knee high', 'uniform', 'long', 'cotton'], [('worn on', 'leg')]]
reflection water water 	 ['reflection', [], [('water', 'water')]]
white light grey grey black standing cow has head 	 ['cow', ['standing', 'black', 'grey', 'light grey', 'white'], [('has', 'head')]]
plastic plastic material bag full of apples filled with apples 	 ['bag', ['plastic material', 'plastic'], [('full of', 'apples'), ('filled with', 'apples')]]
cart wheel wheel wagon 	 ['cart wheel', [], [('wheel', 'wagon')]]
clear white white cloud floating in sky 	 ['cloud', ['white', 'clear white'], [('floating in', 'sky')]]
horses horses outside 	 ['horses', [], [('horses', 'outside')]]
leaves trees trees 	 ['leaves', [], [('trees', 'trees')]]
green grass area area patch grass patch green grass 	 ['grass'

In [42]:
tasks = dict()
for s in ['train', 'val', 'test', 'miniv']:
    tasks[s] = list()

for img_id, img_tasks in vgp.ImgReferTasks.items():
    for task in img_tasks:
        del task['ins_box_ixs']
    split = vgp.vg_loader.info_dict[img_id]['split']
    tasks[split] += img_tasks

for s, ts in tasks.items():
    print(s, len(ts))

train 308893
val 20218
test 18980
miniv 132


In [43]:
import json
all_tasks = list()
for s, ts in tasks.items():
    print(s, len(ts))
    all_tasks += ts
    with open('data/refvg/amt_result/refer_filtered_instance_refine_%s.json' % s, 'w') as f:
        json.dump(ts, f)

print(len(all_tasks))
with open('data/refvg/amt_result/refer_filtered_instance_refine.json', 'w') as f:
        json.dump(all_tasks, f)


train 308893
val 20218
test 18980
miniv 132
348223
